In [1]:
import joblib
import itertools
import pandas as pd 
from arango import ArangoClient
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [2]:
client = ArangoClient(hosts="https://2ae4f052d710.arangodb.cloud:8529")

db = client.db('machine_learning', username='lab_test', password='z-rRdN-Enf4qQwybGiVdbG')

In [3]:
transactions_df = pd.DataFrame(list(db.collection('transactions').all()))
transactions_df['date'] = pd.to_datetime(transactions_df['date'], format='%m/%d/%y')
transactions_df.drop(columns=['_key', '_rev'], inplace=True)
transactions_df.head()

,_id,_from,_to,date,hs6_code,tier
0,transactions/cc90c434a2c8421bb133569d4641dd14,sites/18857ddacc1f473bb65d0a42030601a6,sites/1568051a614a4133b0a36dde7ab1139b,2018-11-30,321410,t1->t0
1,transactions/ac1ff0082e0f4dafa111826516ff2f33,sites/8659f74cc67d4c53b80693eaae86c6f1,sites/4b03dcdf9f89410c8179b5748ac9bb5b,2018-05-02,320890,t1->t0
2,transactions/e943dbf4981849cf8113fcb20003eea6,sites/14c27d3af19a40259c5fbea6c774fd00,sites/9af2b9518dfd4689b1261b84742ee251,2017-05-02,390940,t1->t0
3,transactions/b394774bc9434cdca64e157cc9e29875,sites/540ee69808d3411a8bed3bd349d3f6a4,sites/d77436c4d2c347e2a857b1c2c52f1d91,2016-05-08,731100,t1->t0
4,transactions/8891b03f57ba4d8ca77ba62ea21cd7b2,sites/52b076a7d8c74afaabba4f40af58bf0e,sites/05c982cf09864129913f53372d5eb3fe,2017-05-26,250590,t1->t0


In [4]:
sites_df = pd.DataFrame(list(db.collection('sites').all()))
sites_df.drop(columns=['_key', '_rev'], inplace=True)
sites_df.head()

,_id,organization_id,country_id,effective_closeness,line_rank,page_rank
0,sites/18857ddacc1f473bb65d0a42030601a6,organizations/52c8642c34649eafe2d044eee3d884e1,countries/US,0.0,0.000068,0.000069
1,sites/8659f74cc67d4c53b80693eaae86c6f1,organizations/831a31a1466f0ace3eb20b52d4575f92,countries/Other,1.0,0.000468,0.000089
2,sites/14c27d3af19a40259c5fbea6c774fd00,organizations/f580bc1756d06768c94634b0332e1871,countries/IN,1.0,0.000195,0.000089
3,sites/540ee69808d3411a8bed3bd349d3f6a4,organizations/149081be00548b006dc38a88264eae32,countries/CN,0.0,0.000068,0.000069
4,sites/52b076a7d8c74afaabba4f40af58bf0e,organizations/f4864ac3d5d716cc586d60afe8a403ef,countries/IN,1.0,0.000974,0.000335


In [5]:
countries_df = pd.DataFrame(list(db.collection('countries').all()))
countries_df.drop(columns=['_key', '_rev'], inplace=True)
countries_df.head()

,_id,effective_closeness,line_rank,page_rank
0,countries/US,1.896226,0.217410,0.093527
1,countries/Other,2.037736,0.160780,0.070958
2,countries/IN,1.830189,0.280098,0.121045
3,countries/CN,3.018868,0.018837,0.007930
4,countries/PT,0.000000,0.000602,0.001744


In [6]:
organizations_df = pd.DataFrame(list(db.collection('organizations').all()))
organizations_df.drop(columns=['_key', '_rev'], inplace=True)
organizations_df.head()

,_id,name,effective_closeness,line_rank,page_rank
0,organizations/00014ef830c7109e2fc68dc354d9abb1,GE GRID GMBH,0.000000,0.000071,0.000072
1,organizations/002038373e667f708bfd68c2ab1ea099,Cs Fine Interchem Private Limited,0.000000,0.000071,0.000072
2,organizations/005ceaa6c4d3cf38de771661cb13c8d6,Mahe Neutral Shipping S A S,0.000000,0.000071,0.000072
3,organizations/00737dc27cde70766a18e36f97349bd1,LIAONING RUIXING CHEMICAL GROUP CO,0.000000,0.000071,0.000072
4,organizations/00750bb980757fd8535d73b90d6608c8,Day Glo Color Corp,1.611111,0.004130,0.001880


In [7]:
# joined_df = (
#     transactions_df
#     .merge(sites_df, left_on="_from", right_on="_id", how="left", validate="m:1")
#     .rename(columns={'organization_id':'_from_organization_id', 'country_id':'_from_country_id'})
#     .merge(sites_df, left_on="_to", right_on="_id", how="left", validate="m:1")
#     .rename(columns={'organization_id':'_to_organization_id', 'country_id':'_to_country_id'})
#     .loc[:, ["_from", "_to", "date", "hs6_code", "tier", "_from_organization_id", "_from_country_id", "_to_organization_id", "_to_country_id"]]
# )

# Cohort

In [8]:
print(transactions_df.date.min())
print(transactions_df.date.max())

2016-01-02 00:00:00
2022-07-10 00:00:00


In [9]:
sites_df.shape[0]**2-sites_df.shape[0]

4672082

In [10]:
pairs_df = pd.DataFrame(list(itertools.permutations(sites_df['_id'], 2)), columns=['_from', '_to'])

In [11]:
pairs_df.shape

(4672082, 2)

In [12]:
before_df = transactions_df[(transactions_df['date'] < '2019-01-01')][['_from', '_to']].drop_duplicates(ignore_index=True)
after_df = transactions_df[(transactions_df['date'] >= '2019-01-01')][['_from', '_to']].drop_duplicates(ignore_index=True)

In [15]:
before_merge = pairs_df.merge(before_df, on=['_from', '_to'], validate="1:1", how="left", indicator=True)
after_merge = pairs_df.merge(after_df, on=['_from', '_to'], validate="1:1", how="left", indicator=True)
    
# pairs_df['relationship_before_date'] = (before_merge['_merge'] == 'both')
pairs_df['no_relationship_before_date'] = (before_merge['_merge'] == 'left_only')
# pairs_df['relationship_after_date'] = (after_merge['_merge'] == 'both')
pairs_df['no_relationship_before_but_after'] = (before_merge['_merge'] == 'left_only') & (after_merge['_merge'] == 'both')

In [16]:
cohort_df = pairs_df[pairs_df.no_relationship_before_date][['_from', '_to']].copy()
cohort_df['future_link'] = pairs_df.no_relationship_before_but_after
cohort_df = pd.concat(
    [
        resample(cohort_df[~cohort_df.future_link], replace=False, n_samples=10*len(cohort_df[cohort_df.future_link]), random_state=42),
        cohort_df[cohort_df.future_link]
    ]
)
cohort_df = cohort_df.sample(frac=1).reset_index(drop=True)

In [17]:
cohort_df.future_link.mean()

np.float64(0.09090909090909091)

In [18]:
cohort_df.shape

(7458, 3)

# Feature extraction

In [19]:
features_df = (
    cohort_df.loc[:, ["_from", "_to", "future_link"]]
    .merge(sites_df, left_on="_from", right_on="_id", how="left", validate="m:1")
    .drop(columns=['_id'])
    .rename(columns={'organization_id':'_from_organization_id', 'country_id':'_from_country_id', 'effective_closeness': '_from_site_effective_closeness', 'line_rank': '_from_site_line_rank', 'page_rank': '_from_site_page_rank'})
    .merge(sites_df, left_on="_to", right_on="_id", how="left", validate="m:1")
    .drop(columns=['_id'])
    .rename(columns={'organization_id':'_to_organization_id', 'country_id':'_to_country_id', 'effective_closeness': '_to_site_effective_closeness', 'line_rank': '_to_site_line_rank', 'page_rank': '_to_site_page_rank'})
    .merge(countries_df, left_on="_from_country_id", right_on="_id", how="left", validate="m:1")
    .drop(columns=['_id'])
    .rename(columns={'effective_closeness': '_from_country_effective_closeness', 'line_rank': '_from_country_line_rank', 'page_rank': '_from_country_page_rank'})
    .merge(countries_df, left_on="_to_country_id", right_on="_id", how="left", validate="m:1")
    .drop(columns=['_id'])
    .rename(columns={'effective_closeness': '_to_country_effective_closeness', 'line_rank': '_to_country_line_rank', 'page_rank': '_to_country_page_rank'})
    .merge(organizations_df, left_on="_from_organization_id", right_on="_id", how="left", validate="m:1")
    .drop(columns=['_id', 'name'])
    .rename(columns={'effective_closeness': '_from_organization_effective_closeness', 'line_rank': '_from_organization_line_rank', 'page_rank': '_from_organization_page_rank'})
    .merge(organizations_df, left_on="_to_organization_id", right_on="_id", how="left", validate="m:1")
    .drop(columns=['_id', 'name'])
    .rename(columns={'effective_closeness': '_to_organization_effective_closeness', 'line_rank': '_to_organization_line_rank', 'page_rank': '_to_organization_page_rank'})
)


In [20]:
features_df['min_country_line_rank'] = features_df[['_from_country_line_rank', '_to_country_line_rank']].min(axis=1)
features_df['max_country_line_rank'] = features_df[['_from_country_line_rank', '_to_country_line_rank']].max(axis=1)
features_df['min_country_page_rank'] = features_df[['_from_country_page_rank', '_to_country_page_rank']].min(axis=1)
features_df['max_country_page_rank'] = features_df[['_from_country_page_rank', '_to_country_page_rank']].max(axis=1)
features_df['min_country_effective_closeness'] = features_df[['_from_country_effective_closeness', '_to_country_effective_closeness']].min(axis=1)
features_df['max_country_effective_closeness'] = features_df[['_from_country_effective_closeness', '_to_country_effective_closeness']].max(axis=1)

features_df['min_site_line_rank'] = features_df[['_from_site_line_rank', '_to_site_line_rank']].min(axis=1)
features_df['max_site_line_rank'] = features_df[['_from_site_line_rank', '_to_site_line_rank']].max(axis=1)
features_df['min_site_page_rank'] = features_df[['_from_site_page_rank', '_to_site_page_rank']].min(axis=1)
features_df['max_site_page_rank'] = features_df[['_from_site_page_rank', '_to_site_page_rank']].max(axis=1)
features_df['min_site_effective_closeness'] = features_df[['_from_site_effective_closeness', '_to_site_effective_closeness']].min(axis=1)
features_df['max_site_effective_closeness'] = features_df[['_from_site_effective_closeness', '_to_site_effective_closeness']].max(axis=1)

features_df['min_organization_line_rank'] = features_df[['_from_organization_line_rank', '_to_organization_line_rank']].min(axis=1)
features_df['max_organization_line_rank'] = features_df[['_from_organization_line_rank', '_to_organization_line_rank']].max(axis=1)
features_df['min_organization_page_rank'] = features_df[['_from_organization_page_rank', '_to_organization_page_rank']].min(axis=1)
features_df['max_organization_page_rank'] = features_df[['_from_organization_page_rank', '_to_organization_page_rank']].max(axis=1)
features_df['min_organization_effective_closeness'] = features_df[['_from_organization_effective_closeness', '_to_organization_effective_closeness']].min(axis=1)
features_df['max_organization_effective_closeness'] = features_df[['_from_organization_effective_closeness', '_to_organization_effective_closeness']].max(axis=1)



In [21]:
features_df

,_from,_to,future_link,_from_organization_id,_from_country_id,_from_site_effective_closeness,_from_site_line_rank,_from_site_page_rank,_to_organization_id,_to_country_id,...,min_site_page_rank,max_site_page_rank,min_site_effective_closeness,max_site_effective_closeness,min_organization_line_rank,max_organization_line_rank,min_organization_page_rank,max_organization_page_rank,min_organization_effective_closeness,max_organization_effective_closeness
0,sites/e7f0eff224aa4a30aef3aa8902bc155e,sites/1249c696d89d4d7199847616753b1511,False,organizations/ad4c82787160f622b91b48b277ae0a55,countries/NO,0.000000,0.000137,0.000069,organizations/e87b6063724ec17f0349aa67adcc8feb,countries/CN,...,0.000069,0.000069,0.0,0.000000,0.000071,0.000142,0.000072,0.000072,0.0,0.000000
1,sites/bfb99b32c49d4cb4bcdae7371addb0c2,sites/a4c503f2dd924587a34e8c6adedde6ba,True,organizations/f67ed4e40e995825c159c6bd08b03beb,countries/US,0.000000,0.000068,0.000069,organizations/05c5978192830a0cf35240396987cebd,countries/IN,...,0.000069,0.002610,0.0,1.000000,0.000071,0.006660,0.000072,0.002657,0.0,1.000000
2,sites/19f310b90f7846668cce7d9d3d4a51fc,sites/9058ed6627114e99a820b36ac52f73b6,False,organizations/9b6d01d8339f80bd9d5f141b1e4634e7,countries/CN,0.000000,0.000068,0.000069,organizations/4e4e3d0fbd50e9a372f13aeb220c5eee,countries/Other,...,0.000069,0.000069,0.0,0.000000,0.000071,0.000071,0.000072,0.000072,0.0,0.000000
3,sites/8d86a1958f154fdd93d3bafe1a9cea5b,sites/20432ec1766d4761b7d24d141944efca,False,organizations/f460cbc1fee50b361b095a7e312031b1,countries/HU,0.000000,0.000068,0.000069,organizations/0e12f78cec48d43f0428c515feb0c195,countries/US,...,0.000069,0.000069,0.0,0.000000,0.000071,0.000071,0.000072,0.000072,0.0,0.000000
4,sites/2ed0c1f238e247f1ad3d7d679ea61afd,sites/52527f732f9540b19ddb42191ddc3743,False,organizations/4d4da1d47e1b04f4efb7b794758ca989,countries/CN,0.000000,0.000068,0.000069,organizations/c2b0d39d57d8a97b88cf0360d38dc404,countries/BE,...,0.000069,0.000069,0.0,0.000000,0.000071,0.000071,0.000072,0.000072,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7453,sites/a491dadb6d0247f18b66e85f3d320063,sites/3a510cc4e6c1436bb5164df2ff7f3239,False,organizations/bb9612461fee195badfaf3740ed03ff5,countries/MX,0.000000,0.000068,0.000069,organizations/471c3e055cd2de697d984100274154e0,countries/US,...,0.000069,0.000413,0.0,1.000000,0.000071,0.000994,0.000072,0.000401,0.0,1.000000
7454,sites/b25add42fef14aa4b2ae8d9f634f46d8,sites/2913e97291184f099591e65b4348cec4,False,organizations/eaecc54392624ae9f3765b059cf99f76,countries/Other,0.000000,0.000068,0.000069,organizations/95def8fd465494833cb0934ed32c6c91,countries/FR,...,0.000069,0.000069,0.0,0.000000,0.000071,0.000071,0.000072,0.000072,0.0,0.000000
7455,sites/a88243c022764fbe8da3d3da6769735f,sites/e837cac5407d4df3b2a8222094583322,False,organizations/9ee51a9aeae8e19eacd6373278d29e20,countries/ES,0.000000,0.000068,0.000069,organizations/d44cbeed76fe97fac8b163c2a5fe48b7,countries/US,...,0.000069,0.000069,0.0,0.000000,0.000071,0.000071,0.000072,0.000072,0.0,0.000000
7456,sites/16656ba0c4384a0d95ba6dac2a0686ce,sites/34ef453eb144430aad81dd31b281d61a,False,organizations/0cee83520d779ec5017ba19b7195f970,countries/DE,0.000000,0.000068,0.000069,organizations/07669e3d4eced6dadaffe2f39954f780,countries/DE,...,0.000069,0.000069,0.0,0.000000,0.000071,0.000071,0.000072,0.000072,0.0,0.000000


In [22]:
features = features_df[[
    'min_country_line_rank'
     ,'max_country_line_rank'
     ,'min_country_page_rank'
     ,'max_country_page_rank'
     ,'min_country_effective_closeness'
     ,'max_country_effective_closeness'
     ,'min_site_line_rank'
     ,'max_site_line_rank'
     ,'min_site_page_rank'
     ,'max_site_page_rank'
     ,'min_site_effective_closeness'
     ,'max_site_effective_closeness'
     ,'min_organization_line_rank'
     ,'max_organization_line_rank'
     ,'min_organization_page_rank'
     ,'max_organization_page_rank'
     ,'min_organization_effective_closeness'
     ,'max_organization_effective_closeness']]

target = features_df['future_link']

In [23]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, stratify=target, random_state=42)

In [24]:
clf = RandomForestClassifier(n_estimators=1000, max_depth=5, min_samples_leaf=5, min_samples_split=7, random_state=42, class_weight='balanced')
clf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=5, min_samples_leaf=5,
                       min_samples_split=7, n_estimators=1000, random_state=42)

In [25]:
# Make predictions
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

# Evaluate accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

In [26]:
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

       False       1.00      0.91      0.95      5424
        True       0.53      0.99      0.69       542

    accuracy                           0.92      5966
   macro avg       0.77      0.95      0.82      5966
weighted avg       0.96      0.92      0.93      5966



In [27]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

       False       1.00      0.90      0.95      1356
        True       0.50      0.96      0.65       136

    accuracy                           0.91      1492
   macro avg       0.75      0.93      0.80      1492
weighted avg       0.95      0.91      0.92      1492



In [90]:
joblib.dump(clf, './artifacts/link_prediction_pipeline_1.joblib')

['./artifacts/link_prediction_pipeline_1.joblib']